In [1]:
import torch
import cv2
import csv
import json
import numpy as np
import pandas as pd
# from object_detection import ObjectDetection
import math
import subprocess
from tqdm import tqdm
# import maths

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)
model.eval();

Using cache found in C:\Users\bedo-/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-4-18 Python-3.9.16 torch-2.0.0 CUDA:0 (NVIDIA GeForce GTX 1660, 6144MiB)



requirements: C:\Users\bedo-\.cache\torch\hub\requirements.txt not found, check failed.


Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 


In [13]:
"""
    SORT: A Simple, Online and Realtime Tracker
    Copyright (C) 2016-2020 Alex Bewley alex@bewley.ai

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""
from __future__ import print_function

import os
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io

import glob
import time
import argparse
from filterpy.kalman import KalmanFilter

np.random.seed(0)


def linear_assignment(cost_matrix):
  try:
    import lap
    _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
    return np.array([[y[i],i] for i in x if i >= 0]) #
  except ImportError:
    from scipy.optimize import linear_sum_assignment
    x, y = linear_sum_assignment(cost_matrix)
    return np.array(list(zip(x, y)))


def iou_batch(bb_test, bb_gt):
  """
  From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]
  """
  bb_gt = np.expand_dims(bb_gt, 0)
  bb_test = np.expand_dims(bb_test, 1)
  
  xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
  yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
  xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
  yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
  w = np.maximum(0., xx2 - xx1)
  h = np.maximum(0., yy2 - yy1)
  wh = w * h
  o = wh / ((bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])                                      
    + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1]) - wh)                                              
  return(o)  


def convert_bbox_to_z(bbox):
  """
  Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
  """
  w = bbox[2] - bbox[0]
  h = bbox[3] - bbox[1]
  x = bbox[0] + w/2.
  y = bbox[1] + h/2.
  s = w * h    #scale is just area
  r = w / float(h)
  return np.array([x, y, s, r]).reshape((4, 1))


def convert_x_to_bbox(x,score=None):
  """
  Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
  """
  w = np.sqrt(x[2] * x[3])
  h = x[2] / w
  if(score==None):
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
  else:
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))


class KalmanBoxTracker(object):
  """
  This class represents the internal state of individual tracked objects observed as bbox.
  """
  count = 0
  def __init__(self,bbox):
    """
    Initialises a tracker using initial bounding box.
    """
    #define constant velocity model
    self.kf = KalmanFilter(dim_x=7, dim_z=4) 
    self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
    self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

    self.kf.R[2:,2:] *= 10.
    self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
    self.kf.P *= 10.
    self.kf.Q[-1,-1] *= 0.01
    self.kf.Q[4:,4:] *= 0.01

    self.kf.x[:4] = convert_bbox_to_z(bbox)
    self.time_since_update = 0
    self.id = KalmanBoxTracker.count
    KalmanBoxTracker.count += 1
    self.history = []
    self.hits = 0
    self.hit_streak = 0
    self.age = 0

  def update(self,bbox):
    """
    Updates the state vector with observed bbox.
    """
    self.time_since_update = 0
    self.history = []
    self.hits += 1
    self.hit_streak += 1
    self.kf.update(convert_bbox_to_z(bbox))

  def predict(self):
    """
    Advances the state vector and returns the predicted bounding box estimate.
    """
    if((self.kf.x[6]+self.kf.x[2])<=0):
      self.kf.x[6] *= 0.0
    self.kf.predict()
    self.age += 1
    if(self.time_since_update>0):
      self.hit_streak = 0
    self.time_since_update += 1
    self.history.append(convert_x_to_bbox(self.kf.x))
    return self.history[-1]

  def get_state(self):
    """
    Returns the current bounding box estimate.
    """
    return convert_x_to_bbox(self.kf.x)


def associate_detections_to_trackers(detections,trackers,iou_threshold = 0.3):
  """
  Assigns detections to tracked object (both represented as bounding boxes)

  Returns 3 lists of matches, unmatched_detections and unmatched_trackers
  """
  if(len(trackers)==0):
    return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)

  iou_matrix = iou_batch(detections, trackers)

  if min(iou_matrix.shape) > 0:
    a = (iou_matrix > iou_threshold).astype(np.int32)
    if a.sum(1).max() == 1 and a.sum(0).max() == 1:
        matched_indices = np.stack(np.where(a), axis=1)
    else:
      matched_indices = linear_assignment(-iou_matrix)
  else:
    matched_indices = np.empty(shape=(0,2))

  unmatched_detections = []
  for d, det in enumerate(detections):
    if(d not in matched_indices[:,0]):
      unmatched_detections.append(d)
  unmatched_trackers = []
  for t, trk in enumerate(trackers):
    if(t not in matched_indices[:,1]):
      unmatched_trackers.append(t)

  #filter out matched with low IOU
  matches = []
  for m in matched_indices:
    if(iou_matrix[m[0], m[1]]<iou_threshold):
      unmatched_detections.append(m[0])
      unmatched_trackers.append(m[1])
    else:
      matches.append(m.reshape(1,2))
  if(len(matches)==0):
    matches = np.empty((0,2),dtype=int)
  else:
    matches = np.concatenate(matches,axis=0)

  return matches, np.array(unmatched_detections), np.array(unmatched_trackers)


class Sort(object):
  def __init__(self, max_age=1, min_hits=3, iou_threshold=0.3):
    """
    Sets key parameters for SORT
    """
    self.max_age = max_age
    self.min_hits = min_hits
    self.iou_threshold = iou_threshold
    self.trackers = []
    self.frame_count = 0

  def update(self, dets=np.empty((0, 5))):
    """
    Params:
      dets - a numpy array of detections in the format [[x1,y1,x2,y2,score],[x1,y1,x2,y2,score],...]
    Requires: this method must be called once for each frame even with empty detections (use np.empty((0, 5)) for frames without detections).
    Returns the a similar array, where the last column is the object ID.

    NOTE: The number of objects returned may differ from the number of detections provided.
    """
    self.frame_count += 1
    # get predicted locations from existing trackers.
    trks = np.zeros((len(self.trackers), 5))
    to_del = []
    ret = []
    for t, trk in enumerate(trks):
      pos = self.trackers[t].predict()[0]
      trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]
      if np.any(np.isnan(pos)):
        to_del.append(t)
    trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
    for t in reversed(to_del):
      self.trackers.pop(t)
    matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets,trks, self.iou_threshold)

    # update matched trackers with assigned detections
    for m in matched:
      self.trackers[m[1]].update(dets[m[0], :])

    # create and initialise new trackers for unmatched detections
    for i in unmatched_dets:
        trk = KalmanBoxTracker(dets[i,:])
        self.trackers.append(trk)
    i = len(self.trackers)
    for trk in reversed(self.trackers):
        d = trk.get_state()[0]
        if (trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits):
          ret.append(np.concatenate((d,[trk.id+1])).reshape(1,-1)) # +1 as MOT benchmark requires positive
        i -= 1
        # remove dead tracklet
        if(trk.time_since_update > self.max_age):
          self.trackers.pop(i)
    if(len(ret)>0):
      return np.concatenate(ret)
    return np.empty((0,5))

def parse_args():
    """Parse input arguments."""
    parser = argparse.ArgumentParser(description='SORT demo')
    parser.add_argument('--display', dest='display', help='Display online tracker output (slow) [False]',action='store_true')
    parser.add_argument("--seq_path", help="Path to detections.", type=str, default='data')
    parser.add_argument("--phase", help="Subdirectory in seq_path.", type=str, default='train')
    parser.add_argument("--max_age", 
                        help="Maximum number of frames to keep alive a track without associated detections.", 
                        type=int, default=1)
    parser.add_argument("--min_hits", 
                        help="Minimum number of associated detections before track is initialised.", 
                        type=int, default=3)
    parser.add_argument("--iou_threshold", help="Minimum IOU for match.", type=float, default=0.3)
    args = parser.parse_args()
    return args

if __name__ == '__main__':
    
    video_path = 'light traffic.mp4'
    output_path = 'test.mp4'
    cap = cv2.VideoCapture(video_path)
     #Get the video frame rate and dimensions
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width =  int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height =  int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    ffmpeg_cmd = f"ffmpeg -y -f rawvideo -pix_fmt bgr24 -s {frame_width}x{frame_height} -r {fps} -i - -c:v libx264 -preset fast -crf 30 -pix_fmt nv12 -an -vcodec libx264 {output_path}"

    # Open the output file for writing 
    output_file = subprocess.Popen(ffmpeg_cmd.split(' '), stdin=subprocess.PIPE)
#     if not os.path.exists('output'):
#         os.makedirs('output')
    mot_tracker = Sort() 
    frame_cut = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print('asssss')
            break      

        frame_model = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = model(frame_model[frame_cut:])
#         results = results.xyxy[0][:,:4].cpu()
        track_bbs_ids = mot_tracker.update(results.xyxy[0][:,:4].cpu())
    
        for x1,y1,x2,y2, obj_id in track_bbs_ids:
            print(x1,y1,x2,y2, obj_id)
            cx1 =  int((x1 + x2 ) / 2)
            cy1 =  int((y1 + y2 ) / 2)
            cv2.putText(frame, str(obj_id), (cx1, cy1 - 7), 0, 1, (255, 255, 255), 2)
#             cv2.rectangle(frame, int(x1), int(y1), int(x2), int(y2), (255, 174, 66))

        output_file.stdin.write(frame.tobytes())     
    cap.release()
    output_file.stdin.close()
    output_file.wait()

1367.3381959036108 16.32118589066346 1403.1466673776392 39.76498049117248 9.0
910.9993898426983 4.266929583372277 926.8303220713642 18.809503598390418 8.0
1703.180908613327 62.43408967107503 1757.350829667923 94.76758574396403 7.0
1018.062988593966 32.95485308361814 1041.006835624784 54.26505658435061 6.0
968.3909913672309 34.217550202062725 991.0220945702691 59.940103606531025 5.0
992.9724125625719 66.29486472428526 1027.8165278671156 93.08211891829286 4.0
1772.0476075295883 65.18554347095176 1828.4506834860367 95.14615208568887 3.0
1014.5595088600684 110.30157538148069 1060.2236942649315 154.5990288665662 2.0
1084.4075323208144 107.75067122503341 1134.7692254916856 149.58538834527909 1.0
1105.3471064875075 18.623581869022694 1130.7305302312425 41.03355314074293 10.0
1363.161141172981 16.18676136000441 1399.180492567702 39.399302389253315 9.0
1682.2223954434858 56.85118087919939 1746.605478490606 94.01911098729666 7.0
1018.068726911177 32.97990385268372 1041.950709481948 54.8603356961

1129.1954350851286 25.137868866232417 1157.0191423780377 43.405262590627 12.0
1331.1210033348684 8.05898622061343 1363.5330277288945 29.37404405995233 9.0
1601.1190736114308 42.439280993374744 1645.9166638256993 73.45133496277501 7.0
1031.2535390172702 38.04391573342702 1061.2988509134598 63.86246924367078 6.0
975.1931278533293 40.80676969755419 1005.2930195949366 68.92828854665323 5.0
1003.5980884741316 77.32107519594926 1045.0666948865496 108.04789947181423 4.0
1676.8189395256434 50.01885784934558 1722.7213283011436 73.82753204688119 3.0
1026.4666656623938 145.77659950700024 1082.5402330480613 199.24438629669874 2.0
1107.466572821757 140.5787072564634 1179.3494518224134 195.88205344953653 1.0
1131.1376021827393 25.158176192323765 1158.1710547770288 42.87798757399287 12.0
1597.7383341625593 41.994881706098994 1640.673273059477 72.74380101647877 7.0
1033.1262538616777 38.175209489736496 1063.3414042395304 63.56903079596281 6.0
974.996097307606 42.02642568826316 1005.4826781880928 69.29

969.6509457551732 0.7033380187954457 990.7642926492794 18.387488464796913 18.0
1512.6097323638387 26.822021332308218 1566.5283493090496 55.66953448051182 7.0
1050.292779188877 49.92257740998332 1085.7380478175469 77.84296876068143 6.0
984.5715619820138 50.6189103352075 1019.9313834937565 80.91235087897358 5.0
1017.8479557796455 96.2798158097187 1067.469863018154 138.84995036022622 4.0
1040.8330045735163 212.62193018248647 1118.685807840712 287.8818494424902 2.0
1167.9784684182657 204.85344303893572 1258.8093555113435 280.88822024435024 1.0
969.8701132074301 0.42962887035215935 991.222621042565 18.253529282367168 18.0
1507.3946836551577 26.099207964233074 1560.7634900126468 54.115511815581215 7.0
1050.9223592496637 50.70685106952111 1086.7328730786326 78.67547163950543 6.0
986.1888467133227 50.89356672565903 1021.3158713146046 81.28098647243128 5.0
1017.7438181769418 97.95355116781488 1068.3156667883673 141.22879185757444 4.0
1041.3973092940537 217.04669659012333 1120.6676180817367 293.

925.6157130269826 3.3408064506760127 952.643509159821 24.526041759052877 22.0
1506.3198641027486 22.527656209943935 1542.0329465723187 41.5260394044921 19.0
976.6323391130402 1.1268985061388435 998.8634934486117 21.526615657670288 18.0
1426.6331127007873 16.44760636046938 1483.0596339621754 39.5778171026202 7.0
1072.160231897507 68.40971577273098 1111.043547436002 100.49810544798368 6.0
992.525052614491 69.36060371963802 1032.515019970201 102.32188649250804 5.0
1029.6869548956884 134.79135153830845 1092.311433943678 189.26943226459372 4.0
1062.8788084488338 384.82376710159593 1192.6582369149432 525.4998793755642 2.0
1272.795932822798 371.102929366931 1424.8241572138654 506.6864624150428 1.0
925.4715576891264 3.3920086900525543 952.8700515242248 24.78103557155304 22.0
1504.7434028816945 22.239613190690907 1540.0539487544427 41.21186349075073 19.0
976.5921678883217 1.1011654018123416 999.2100194972816 21.76019908551022 18.0
1425.377225648246 16.229603960126123 1481.123238745992 38.983414

1224.0503283107425 48.285101387716004 1264.3278061470203 74.76196878075277 26.0
926.4722956896851 4.33476587183495 954.7011136951548 26.738274530585986 22.0
981.7176423835947 3.388828429643338 1010.1042314997546 24.018346194669533 18.0
1366.8054644322503 10.43591631088792 1398.2401221343569 27.50798927681077 7.0
1095.1925892901804 86.63295299980372 1142.1269796404808 126.12980081459905 6.0
998.8891782693182 91.35132607617778 1045.4816638609625 130.65037903014203 5.0
1038.9018082293521 191.77212298515974 1119.9191858327993 265.0137016153457 4.0
1099.6522314779038 849.8218337369982 1368.9288981375123 1114.8746781049051 2.0
1509.826558139934 805.906294256223 1878.752628119303 1101.1540043328087 1.0
1226.4551531245315 49.54434118898581 1267.0425080178052 76.10670705100294 26.0
927.1466394985952 4.175093560921393 955.6409553852264 26.523047522989568 22.0
981.2326808539625 3.282508232987757 1010.2836942763605 24.431149102077867 18.0
1361.982779564717 9.381964178450174 1393.4080592251585 26.6

...

1820.095072719933 36.828573331204346 1891.2422037190909 72.94628593997757 73.0
1628.062691255217 19.973179452098844 1673.1986423029985 42.67297066267416 72.0
877.580718414175 0.1767983639248829 902.0097157829912 20.346163726186305 71.0
1259.4899327099922 42.77145683620525 1316.0029846730251 87.34133856760405 68.0
956.6980409551436 12.305263207510414 989.4578415734168 46.32930374612332 63.0
862.2165418475047 26.60141169202505 896.4244558515111 55.648828324753495 62.0
1458.4525219113893 3.6740711842463014 1496.9409623125885 30.40373392718191 60.0
716.7820022139617 335.54376127294825 850.1577897370801 474.1134352322964 55.0
1017.0389685579767 162.15050792907832 1094.2164132059097 232.74596986449782 54.0
783.3365578393891 129.26824091974572 843.3602805878322 186.27055843814838 38.0
1809.4366224615098 34.310789676962294 1879.8711813015773 70.89327638019398 73.0
1622.6007269327029 18.749940799878672 1667.3119834783952 41.382210372634674 72.0
878.1447275867091 0.24969942896177955 902.92840385

1338.6525883065922 43.94119122588937 1391.4025397115822 85.64663577690094 76.0
1724.784561150155 25.421023975532204 1786.779614424088 57.537854649382254 73.0
873.8819834649807 0.8197648601539385 902.7180523642774 25.43071190263121 71.0
1305.8041927342754 67.24893346018932 1367.1112944716363 106.91097571295204 68.0
964.3431761831397 19.358244779779774 999.7388507053348 55.45593251479642 63.0
859.3733866431027 38.24288550244848 893.8104246604129 66.63998208480443 62.0
1413.2938915699322 -0.5529255350272919 1448.361411860925 22.80562024909458 60.0
504.1654703964086 579.008857557814 732.4983524220903 840.3152568846774 55.0
1031.3668849640592 225.44524731093955 1128.8358998798074 320.1653434767832 54.0
742.7126362088356 173.10628916145183 820.4142684665112 243.314109335781 38.0
1343.341309827455 46.88039617670475 1397.1790503514612 88.99171772915179 76.0
1715.8122678289521 24.93333011761169 1777.2089276229688 56.09124088738318 73.0
874.0901295777827 1.478418067960833 903.3462115042408 26.10

1453.403925340969 75.52899114911408 1529.0050795618333 131.34385995584603 76.0
1616.6888642707236 13.352140788567281 1666.071379870045 40.77153820907526 73.0
869.72099102213 1.6839776768904429 904.6667319284486 36.219632518362566 71.0
1394.3806468133296 97.26657988757583 1476.5910081132813 150.92827208797723 68.0
977.7712042553861 30.64059049574955 1016.8323981539305 75.92433304366116 63.0
851.3940588503874 49.323418378702996 887.044607820083 85.69420992099447 62.0
1056.0770922830222 418.1291699961172 1221.6582051736257 602.6702976828736 54.0
625.7729402712321 283.1898623909464 748.184692621704 402.1115641872979 38.0
1460.3118033051323 77.4215857145519 1537.8754408160635 135.137762423026 76.0
1611.649788145991 13.275268537606292 1660.3291657771324 40.85695251624547 73.0
869.4500320301618 2.2047250514444485 904.3398948704839 37.38341240886817 71.0
1401.8294057145333 100.07795019495983 1484.9676878104476 155.20947767323244 68.0
978.3115176066611 31.00228518158148 1017.2271725596369 77.19

988.281097989193 37.78051421003327 1021.1638506170008 67.84882050368586 90.0
1778.5903427265323 36.654498292660534 1844.639861971703 73.81501805245415 83.0
1669.8066881507818 137.3312636932845 1788.170780706191 223.51670040697823 76.0
1528.1463441476587 3.9850052827773634 1573.200988386361 34.26916060532509 73.0
863.8626807755569 12.827172614591355 902.857870001861 47.66615950901762 71.0
1545.1722356173887 155.6776818747535 1661.1689639032274 234.00955306709324 68.0
993.7820462993463 64.95249224178292 1033.876520511637 104.39107517169663 63.0
834.1309435334039 77.04895299488108 877.016968365862 114.31645670793436 62.0
264.2188520972138 611.8830140022596 532.0347252042475 902.776817027951 38.0
988.6495900500671 38.204703675760975 1021.1252668352278 67.57540561438134 90.0
1775.2733192399771 36.42099179130638 1840.8065886548222 73.02557114617423 83.0
1676.5156651316486 139.22466000044017 1797.9714426264097 227.95360718879374 76.0
1526.4714760919978 3.824746113230905 1571.0454629153846 33.

1619.5439978690433 180.93803694134004 1919.9454048156094 277.4497633774122 94.0
990.9230293587065 45.74555194833663 1036.569667285426 97.30714747310262 90.0
1730.4783389817403 32.99787241612035 1786.9069456184252 68.16753582860906 83.0
1791.3660437412032 175.57271447920868 1924.566704812055 279.54732473576667 76.0
1504.9582805127275 3.2108734865658075 1544.9946309501865 25.228119803408354 73.0
862.509579620798 18.51198350420201 901.0010786309075 50.02268030198668 71.0
1623.8070283373959 188.57584988273598 1769.1139951461964 285.7438157295884 68.0
996.005253500137 74.17754399811417 1039.626095937081 116.70611431537344 63.0
830.4941265920723 86.63731264486408 874.6730135806362 128.3442637753663 62.0
-4.483771824289306 893.1272073845704 357.395751568534 1087.260653374908 38.0
1619.5575833856374 180.93640434158385 1919.9532797215147 277.4345017181533 94.0
990.9162600200026 45.75007423041741 1036.5200858828289 97.26980336999397 90.0
1730.6822803837397 33.02391725027184 1787.0493131111318 68

In [ ]:
results.xyxy[0][:,:4]

In [7]:
for x1,y1,x2,y2, obj_id in track_bbs_ids:
    print(x1,y1,x2,y2, obj_id)

1367.3381959036108 16.32118589066346 1403.1466673776392 39.76498049117248 9.0
910.9993898426983 4.266929583372277 926.8303220713642 18.809503598390418 8.0
1703.180908613327 62.43408967107503 1757.350829667923 94.76758574396403 7.0
1018.062988593966 32.95485308361814 1041.006835624784 54.26505658435061 6.0
968.3909913672309 34.217550202062725 991.0220945702691 59.940103606531025 5.0
992.9724125625719 66.29486472428526 1027.8165278671156 93.08211891829286 4.0
1772.0476075295883 65.18554347095176 1828.4506834860367 95.14615208568887 3.0
1014.5595088600684 110.30157538148069 1060.2236942649315 154.5990288665662 2.0
1084.4075323208144 107.75067122503341 1134.7692254916856 149.58538834527909 1.0
